# Using QEPy with Envyron
## Simulations of Isolated Systems Under External Pressure

In [ ]:
import qepy
import importlib

In [ ]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [ ]:
from ase import Atoms
import matplotlib.pyplot as plt

## Set System and QE Parameters

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./data/pseudo/'"
    },
    '&system': {
        'ecutrho' : 150,
        'ecutwfc' : 30,
        'ibrav' : 0,
        'nat' : 3,
        'ntyp' : 2,
        'ibrav' : 0
    },
    '&electrons': {
        'conv_thr' : 1e-10,
        'diagonalization' : "'cg'",
        'mixing_beta' : 0.4,
        'electron_maxstep' : 200
    },
    'atomic_positions bohr': ['O   6.79  7.05  6.50','H   8.45  6.22  6.50','H   5.56  5.66  6.50'],
    'atomic_species': ['H   1.  H.pbe-rrkjus.UPF','O  16.  O.pbe-rrkjus.UPF'],
    'k_points automatic': ['1 1 1 0 0 0'],
    'cell_parameters bohr':[
        '15.  0.    0.',
        '0.  15.    0.',
        '0.   0.   15.'],
}

In [ ]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

Run the SCF loop in the notebook

In [ ]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

## Setup Environment and Calculator

Extract ASE atoms from the QEPy driver

In [ ]:
atoms = driver.get_ase_atoms()

Convert ASE atoms data into the quantities expected by Environ

NOTE: valence charges need to be extracted from the QEpy driver

In [ ]:
natoms = len(atoms.numbers)
ntypes = len(np.unique(atoms.numbers))
ion_ids = list(np.unique(atoms.numbers))
ion_labels = list(np.unique(atoms.get_chemical_symbols()))
ion_weigths = list(np.unique(atoms.get_masses()))
itypes = [ ion_ids.index(id) for id in atoms.numbers]
zv = list(driver.qepy.ions_base.get_array_zv()[:ntypes])
coords = atoms.positions / 0.52917720859

Generate an Environ grid extracting information on the cell and grid from the driver

In [ ]:
from envyron.domains import EnvironGrid
at = driver.get_ions_lattice()
nr = driver.get_number_of_grid_points()
grid = EnvironGrid(at, nr, label='system')

In [ ]:
from envyron.representations import EnvironDensity
rho = EnvironDensity(grid)

Read Environ input

In [ ]:
from envyron.io.input import Input
my_input = Input(natoms=natoms, filename='data/volume.yml')

Create the Environ Setup 

In [ ]:
from envyron.setup import Setup
my_setup = Setup(my_input)
my_setup.init_cell(grid)
my_setup.init_numerical(False)

Create the Environ Object

In [ ]:
from envyron.main import Main
environ = Main(my_setup,natoms,ntypes,itypes,zv,ion_ids)
environ.update_cell_dependent_quantities()
environ.update_ions(coords)

Setup the Environ Calculator

In [ ]:
from envyron.calculator import Calculator
my_calculator = Calculator(environ)

## Run the Calculation with Environ

Restart a new driver

In [ ]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

In [ ]:
volumes = []
for i in range(60):

    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

    # pass new electronic density to Environ
    rho = driver.data2field(driver.get_density().copy())
    environ.update_electrons(rho)
    volumes.append(environ.solvent.volume)
    # compute Environ contribution to the energy
    my_calculator.energy()
    driver.embed.extene = environ.evolume
    my_calculator.potential(True)
    driver.set_external_potential(driver.field2data(environ.vsoftcavity), exttype=0)

etot = driver.calc_energy()
print(etot, environ.evolume, etot + environ.evolume)